This is a demonstration of how the datasets could be utilized to compute Residual Volatility (IVOL) factor ranks

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import re
import json
import requests

directory = os.fsencode('../data')
res_series = []
beta_series = []

for file in os.scandir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.json'):
        try:
            with open(filename) as f:
                index = re.search('(?<=-).+(?= )', f.name).group()  # extract date
                data = json.load(f)
                # backwards compatiability for data sets that didn't have residuals calculated
                if 'residuals' in data:
                    res_json_series = pd.Series(data['residuals'])
                    beta_json_series = pd.Series(data['values'])
                    res_series.append(res_json_series.rename(index))
                    beta_series.append(beta_json_series.rename(index))
        except ValueError:
            continue

res_df = pd.DataFrame(res_series).sort_index()
res_df.index = pd.to_datetime(res_df.index)
betas_df = pd.DataFrame(beta_series).sort_index()
betas_df.index = pd.to_datetime(betas_df.index)

df_monthly = res_df.resample('ME').std() # convert to IVOL
upper_ivols = df_monthly.apply(lambda x: x[x >= x.quantile(.8)], axis=1) # upper quintile IVOL
bottom_ivols = df_monthly.apply(lambda x: x[x <= x.quantile(.25)], axis=1) # bottom quintile IVOL

bottom_ivols

Now I'll scrape CBOE weekly options CSV for tickers and intersect them with the top and bottom quartiles of the IVOL portfolio

In [2]:
cboe_csv_link = 'https://www.cboe.com/available_weeklys/get_csv_download/'
output = requests.get(cboe_csv_link).text

find_str = "Available Weeklys - Exchange Traded Products (ETFs and ETNs)"

idx = output.find(find_str)

skiprows_val = output[:idx+len(find_str)].count("\n")


cboe_csv = pd.read_csv(cboe_csv_link, skiprows=skiprows_val, usecols=[0], header=None)
tickers_df = cboe_csv[(cboe_csv[0] != 'Available Weeklys - Exchange Traded Products (ETFs and ETNs)')
                      & (cboe_csv[0] != 'Available Weeklys - Equity')]

weekly_option_tickers = tickers_df[0]

recent_upper_ivols = upper_ivols.iloc[-1]
upper_ivol_tickers = recent_upper_ivols[recent_upper_ivols.notna()].index
recent_bottom_ivols = bottom_ivols.iloc[-1]
bottom_ivol_tickers = recent_bottom_ivols[recent_bottom_ivols.notna()].index

optionable_upper_ivol_tickers = weekly_option_tickers[weekly_option_tickers.isin(upper_ivol_tickers)]
optionable_bottom_ivol_tickers = weekly_option_tickers[weekly_option_tickers.isin(bottom_ivol_tickers)]

Now, I want to intersect further with the lower quartile of the low beta portfolio from the recent month.

In [4]:
low_beta = betas_df.resample('ME').mean().apply(lambda x: x[x <= x.quantile(.25)], axis=1)
recent_low_betas = low_beta.iloc[-1]
low_beta_tickers = recent_low_betas[recent_low_betas.notna()].index
optionable_bottom_ivol_tickers[optionable_bottom_ivol_tickers.isin(low_beta_tickers)]

110     ADP
116    AGNC
169     BMY
171     BSX
180     CAG
181     CAH
187    CBOE
196    CHTR
198      CI
200      CL
205     CLX
216     CPB
243      DG
258    EBAY
268     EPD
270      ET
284    FOXA
291      GD
298    GILD
325     HRL
353       K
358     KMB
371     LMT
389     MCK
391    MDLZ
402      MO
406     MRK
428     NOC
460     PEP
462      PG
469      PM
493     RTX
530     STZ
534       T
547     TMO
567     UNH
575       V
596      WM
598     WMT
Name: 0, dtype: object

These are all presumably safe stocks to long. I could create carry by short selling puts, trade their skew, or even make a dispersion trade by shorting options against the opposite leg.